In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import jaccard_score
from scipy.spatial.distance import pdist, squareform

In [2]:
data = pd.read_csv('./Dataset .csv')

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

In [4]:
data.isnull().sum()

Restaurant ID           0
Restaurant Name         0
Country Code            0
City                    0
Address                 0
Locality                0
Locality Verbose        0
Longitude               0
Latitude                0
Cuisines                9
Average Cost for two    0
Currency                0
Has Table booking       0
Has Online delivery     0
Is delivering now       0
Switch to order menu    0
Price range             0
Aggregate rating        0
Rating color            0
Rating text             0
Votes                   0
dtype: int64

In [5]:
data = data.dropna()

In [6]:
df = data[['Restaurant ID','Restaurant Name','City','Address','Cuisines','Average Cost for two','Votes','Aggregate rating']]

In [7]:
df = df.rename(columns={'Restaurant ID':'id','Restaurant Name':'restaurant name','City':'city','Address':'address',
                       'Cuisines':'cusines','Average Cost for two':'avg cost','Votes':'votes','Aggregate rating':'rating'})

In [8]:
df.head()

,id,restaurant name,city,address,cusines,avg cost,votes,rating
0,6317637,Le Petit Souffle,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","French, Japanese, Desserts",1100,314,4.8
1,6304287,Izakaya Kikufuji,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...",Japanese,1200,591,4.5
2,6300002,Heat - Edsa Shangri-La,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Seafood, Asian, Filipino, Indian",4000,270,4.4
3,6318506,Ooma,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","Japanese, Sushi",1500,365,4.9
4,6314302,Sambo Kojin,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","Japanese, Korean",1500,229,4.8


In [9]:
df.duplicated().sum()

0

In [10]:
df = df.sort_values(by='rating',ascending=False)
df

,id,restaurant name,city,address,cusines,avg cost,votes,rating
1381,18384115,Caterspoint,Gurgaon,"S-27/11, DLF Phase 3, Gurgaon","Mexican, American, Healthy Food",500,223,4.9
9296,7417450,Talaga Sampireun,Tangerang,"Jl. Boulevard Bintaro Jaya Blok B7/N1, Bintaro...","Sunda, Indonesian",200000,2212,4.9
9303,18217279,Miann,Auckland,"57 Fort Street, Auckland Auckland CBD",Desserts,25,281,4.9
9299,7001086,Milse,Auckland,"The Pavilions, 27 Tyler Street, Britomart, Auc...",Desserts,50,754,4.9
428,17142792,Mama's Fish House,Rest of Hawaii,"799 Poho Pl, Paia, HI 96779","Hawaiian, Seafood",70,1343,4.9
...,...,...,...,...,...,...,...,...
5785,313163,A Patisseries Gallery,New Delhi,"H-53, Naraina, New Delhi","Bakery, Desserts",200,0,0.0
5786,18398504,Aggarwal Bikaneri Sweets,New Delhi,"WZ-258, Main Market, Near Pusa Gate, Inderpuri...",Mithai,100,0,0.0
5787,301932,Aggarwal Sweets & Bakers,New Delhi,"Ring Road, Naraina, New Delhi","Desserts, Street Food",200,1,0.0
5788,18312572,Avatar Da Dhaba,New Delhi,"Ring Road Narayna, Opp Dharamshala, Naraina, N...",North Indian,200,0,0.0


In [11]:
df = df[(df['votes']>10) & (df['rating']>3)]
df

,id,restaurant name,city,address,cusines,avg cost,votes,rating
1381,18384115,Caterspoint,Gurgaon,"S-27/11, DLF Phase 3, Gurgaon","Mexican, American, Healthy Food",500,223,4.9
9296,7417450,Talaga Sampireun,Tangerang,"Jl. Boulevard Bintaro Jaya Blok B7/N1, Bintaro...","Sunda, Indonesian",200000,2212,4.9
9303,18217279,Miann,Auckland,"57 Fort Street, Auckland Auckland CBD",Desserts,25,281,4.9
9299,7001086,Milse,Auckland,"The Pavilions, 27 Tyler Street, Britomart, Auc...",Desserts,50,754,4.9
428,17142792,Mama's Fish House,Rest of Hawaii,"799 Poho Pl, Paia, HI 96779","Hawaiian, Seafood",70,1343,4.9
...,...,...,...,...,...,...,...,...
1028,9224,Kay's Bake Land,Faridabad,"14, Huda Market, Sector 37, Faridabad","Bakery, Desserts",150,19,3.1
1650,18264977,Giani's,Gurgaon,"Shop 3, Shri Ram Mandir Complex, Palam Vihar, ...","Ice Cream, Desserts",400,24,3.1
4568,9459,Gulnar Bar Be Que,New Delhi,"7-A/45, WEA, Karol Bagh, New Delhi","North Indian, Chinese",700,73,3.1
6796,304186,Hangchuaa's Chinese Food Corner,New Delhi,"D-65, Arjun Nagar, Safdarjung Enclave, Safdarj...",Chinese,600,35,3.1


In [12]:
df['cusines'] = df['cusines'].str.split(', ')
df

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_11796\3312058883.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cusines'] = df['cusines'].str.split(', ')


,id,restaurant name,city,address,cusines,avg cost,votes,rating
1381,18384115,Caterspoint,Gurgaon,"S-27/11, DLF Phase 3, Gurgaon","[Mexican, American, Healthy Food]",500,223,4.9
9296,7417450,Talaga Sampireun,Tangerang,"Jl. Boulevard Bintaro Jaya Blok B7/N1, Bintaro...","[Sunda, Indonesian]",200000,2212,4.9
9303,18217279,Miann,Auckland,"57 Fort Street, Auckland Auckland CBD",[Desserts],25,281,4.9
9299,7001086,Milse,Auckland,"The Pavilions, 27 Tyler Street, Britomart, Auc...",[Desserts],50,754,4.9
428,17142792,Mama's Fish House,Rest of Hawaii,"799 Poho Pl, Paia, HI 96779","[Hawaiian, Seafood]",70,1343,4.9
...,...,...,...,...,...,...,...,...
1028,9224,Kay's Bake Land,Faridabad,"14, Huda Market, Sector 37, Faridabad","[Bakery, Desserts]",150,19,3.1
1650,18264977,Giani's,Gurgaon,"Shop 3, Shri Ram Mandir Complex, Palam Vihar, ...","[Ice Cream, Desserts]",400,24,3.1
4568,9459,Gulnar Bar Be Que,New Delhi,"7-A/45, WEA, Karol Bagh, New Delhi","[North Indian, Chinese]",700,73,3.1
6796,304186,Hangchuaa's Chinese Food Corner,New Delhi,"D-65, Arjun Nagar, Safdarjung Enclave, Safdarj...",[Chinese],600,35,3.1


In [13]:
df = df.explode('cusines')
df

,id,restaurant name,city,address,cusines,avg cost,votes,rating
1381,18384115,Caterspoint,Gurgaon,"S-27/11, DLF Phase 3, Gurgaon",Mexican,500,223,4.9
1381,18384115,Caterspoint,Gurgaon,"S-27/11, DLF Phase 3, Gurgaon",American,500,223,4.9
1381,18384115,Caterspoint,Gurgaon,"S-27/11, DLF Phase 3, Gurgaon",Healthy Food,500,223,4.9
9296,7417450,Talaga Sampireun,Tangerang,"Jl. Boulevard Bintaro Jaya Blok B7/N1, Bintaro...",Sunda,200000,2212,4.9
9296,7417450,Talaga Sampireun,Tangerang,"Jl. Boulevard Bintaro Jaya Blok B7/N1, Bintaro...",Indonesian,200000,2212,4.9
...,...,...,...,...,...,...,...,...
1650,18264977,Giani's,Gurgaon,"Shop 3, Shri Ram Mandir Complex, Palam Vihar, ...",Desserts,400,24,3.1
4568,9459,Gulnar Bar Be Que,New Delhi,"7-A/45, WEA, Karol Bagh, New Delhi",North Indian,700,73,3.1
4568,9459,Gulnar Bar Be Que,New Delhi,"7-A/45, WEA, Karol Bagh, New Delhi",Chinese,700,73,3.1
6796,304186,Hangchuaa's Chinese Food Corner,New Delhi,"D-65, Arjun Nagar, Safdarjung Enclave, Safdarj...",Chinese,600,35,3.1


In [14]:
xtab = pd.crosstab(df['restaurant name'],
                                df['cusines'])
xtab

cusines,Afghani,African,American,Andhra,Arabian,Argentine,Armenian,Asian,Asian Fusion,Assamese,...,Teriyaki,Tex-Mex,Thai,Tibetan,Turkish,Turkish Pizza,Vegetarian,Vietnamese,Western,World Cuisine
restaurant name,,,,,,,,,,,,,,,,,,,,,
#45,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#Dilliwaala6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#OFF Campus,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
#Urban Caf��,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
'Ohana,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
t Lounge by Dilmah,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tashas,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
wagamama,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
jaccard_dist = pdist(xtab.values, metric='jaccard')
jaccard_matrix = squareform(jaccard_dist)
jaccard_sim = 1 - jaccard_matrix
df_jaccard = pd.DataFrame(
    jaccard_sim,
    index=xtab.index,
    columns=xtab.index)

df_jaccard

restaurant name,#45,#Dilliwaala6,#OFF Campus,#Urban Caf��,'Ohana,10 Downing Street,10 To 10 In Delhi,11th Avenue Cafe Bistro,12212,145 Kala Ghoda,...,d' Curry House,eastWEST - Radisson Hotel,feel ALIVE,hug!,sketch Gallery,t Lounge by Dilmah,tashas,wagamama,{Niche} - Cafe & Bar,�ukura��a Sofras۱
restaurant name,,,,,,,,,,,,,,,,,,,,,
#45,1.000000,0.000000,0.250000,0.000000,0.0,0.0,0.500000,0.250000,0.00,0.000000,...,0.000000,0.333333,0.000000,0.000000,0.0,0.333333,0.50,0.0,0.000000,0.0
#Dilliwaala6,0.000000,1.000000,0.000000,0.333333,0.0,0.0,0.000000,0.000000,0.00,0.000000,...,0.250000,0.333333,0.250000,0.000000,0.0,0.000000,0.00,0.0,0.250000,0.0
#OFF Campus,0.250000,0.000000,1.000000,0.166667,0.0,0.0,0.200000,0.600000,0.25,0.166667,...,0.142857,0.400000,0.000000,0.000000,0.0,0.166667,0.20,0.0,0.333333,0.0
#Urban Caf��,0.000000,0.333333,0.166667,1.000000,0.0,0.0,0.000000,0.166667,0.00,0.000000,...,0.400000,0.200000,0.166667,0.000000,0.0,0.000000,0.00,0.0,0.750000,0.0
'Ohana,0.000000,0.000000,0.000000,0.000000,1.0,0.0,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
t Lounge by Dilmah,0.333333,0.000000,0.166667,0.000000,0.0,0.0,0.250000,0.166667,0.00,0.200000,...,0.000000,0.200000,0.000000,0.333333,0.0,1.000000,0.25,0.0,0.000000,0.0
tashas,0.500000,0.000000,0.200000,0.000000,0.0,0.0,0.333333,0.200000,0.00,0.000000,...,0.000000,0.250000,0.000000,0.000000,0.0,0.250000,1.00,0.0,0.000000,0.0
wagamama,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.00,0.000000,...,0.000000,0.000000,0.200000,0.000000,0.0,0.000000,0.00,1.0,0.000000,0.0


In [16]:
rest_name = "Gulnar Bar Be Que"

sim = df_jaccard.loc[rest_name].sort_values(ascending=False)

sim = pd.DataFrame({'restaurant name': sim.index, 'sim_score': sim.values})
sim = sim[(sim['restaurant name']!= rest_name) & (sim['sim_score']>=0.7)].head(5)


res_rec = pd.merge(sim,df[['restaurant name','rating','city','address','cusines']],how='inner',on='restaurant name')
res_rec_final = res_rec.sort_values('rating',ascending=False).drop_duplicates('restaurant name',keep='first')

In [17]:
res_rec_final

,restaurant name,sim_score,rating,city,address,cusines
6,Spicy Tadka,1.0,3.5,New Delhi,"BE 332, Hari Nagar Street 3, Mayapuri Phase 2,...",North Indian
4,The Cinnamon Kitchen,1.0,3.4,Gurgaon,"Plot 391, Saraswati Kunj, Sector 53, Golf Cour...",North Indian
0,Halki Aanch,1.0,3.3,Gurgaon,"D-141, Shopping Mall, Arjun Marg, DLF Phase 1,...",North Indian
8,Desi Jugaad,1.0,3.2,New Delhi,"G-6, Plot 14, Vardhaman Plaza, Mayur Vihar Pha...",North Indian
2,Take N Taste,1.0,3.1,New Delhi,"3/30, Double Storey, Vijay Nagar, New Delhi",North Indian
